#####Invocando librerias y funciones necesarias

In [0]:
from pyspark.sql.functions import col, lit, struct, current_timestamp, current_date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType

#####Leyendo archivos .json de las carpetas del DBFS

In [0]:
app_df = spark.read.json("/temp/trx_app/*.json")
web_df = spark.read.json("/temp/trx_web/*.json")
atm_df = spark.read.json("/temp/trx_atm/*.json")

#####Reestructurando la información contenida en cada archivo .json para que tengan las misma  
#####estructura y luego poder unirlos en un solo dataframe

In [0]:
app_df_agg = app_df.withColumn("transaccion",col("transaccion").withField(
                                                                "detalles_web",struct(
                                                                    lit("").cast("string").alias("ip_origen"),
                                                                    lit("").cast("string").alias("navegador"),
                                                                    lit("").cast("string").alias("dispositivo_web")         
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "ubicacion_atm",struct(
                                                                    lit("").cast("string").alias("ciudad"), 
                                                                    lit("").cast("string").alias("direccion"),
                                                                    lit("").cast("string").alias("codigo_atm")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "tarjeta",struct(
                                                                    lit("").cast("string").alias("numero_tarjeta"),
                                                                    lit("").cast("string").alias("titular_atm")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "autenticacion",struct(
                                                                    col("transaccion.autenticacion.metodo"),
                                                                    col("transaccion.autenticacion.exitoso"),
                                                                    lit("").cast("string").alias("codigo_enviado")
                                                                    )
                                                                )

                    ) 

web_df_agg = web_df.withColumn("transaccion", col("transaccion").withField(
                                                                "detalles_app",struct(
                                                                    lit("").cast("string").alias("dispositivo_app"),
                                                                    lit("").cast("string").alias("sistema_operativo"),
                                                                    lit("").cast("string").alias("version_app")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion", col("transaccion").withField(
                                                                "ubicacion",struct(
                                                                    lit("").cast("double").alias("latitud"),
                                                                    lit("").cast("double").alias("longitud")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "ubicacion_atm",struct(
                                                                    lit("").cast("string").alias("ciudad"), 
                                                                    lit("").cast("string").alias("direccion"),
                                                                    lit("").cast("string").alias("codigo_atm")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "tarjeta",struct(
                                                                    lit("").cast("string").alias("numero_tarjeta"),
                                                                    lit("").cast("string").alias("titular_atm")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "autenticacion",struct(
                                                                    col("transaccion.autenticacion.metodo"),
                                                                    col("transaccion.autenticacion.codigo_enviado"),
                                                                    lit("").cast("string").alias("exitoso")
                                                                    )
                                                                )
                    )

atm_df_agg = atm_df.withColumn("transaccion", col("transaccion").withField(
                                                                "detalles_app",struct(
                                                                    lit("").cast("string").alias("dispositivo_app"),
                                                                    lit("").cast("string").alias("sistema_operativo"),
                                                                    lit("").cast("string").alias("version_app")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion",col("transaccion").withField(
                                                                "detalles_web",struct(
                                                                    lit("").cast("string").alias("ip_origen"),
                                                                    lit("").cast("string").alias("navegador"),
                                                                    lit("").cast("string").alias("dispositivo_web")         
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion", col("transaccion").withField(
                                                                "ubicacion",struct(
                                                                    lit("").cast("double").alias("latitud"),
                                                                    lit("").cast("double").alias("longitud")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion", col("transaccion").withField(
                                                                "autenticacion",struct(
                                                                    lit("").cast("string").alias("metodo"),
                                                                    lit("").cast("string").alias("exitoso"),
                                                                    lit("").cast("string").alias("codigo_enviado")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion", col("transaccion").withField(
                                                                "cuenta_destino_app",struct(
                                                                    lit("").cast("string").alias("numero_cuenta_app"),
                                                                    lit("").cast("string").alias("banco_app"),
                                                                    lit("").cast("string").alias("titular_app")
                                                                    )
                                                                )
                    ) \
                    .withColumn("transaccion", col("transaccion").withField(
                                                                "cuenta_destino_web",struct(
                                                                    lit("").cast("string").alias("numero_cuenta_app"),
                                                                    lit("").cast("string").alias("banco_app"),
                                                                    lit("").cast("string").alias("titular_app")
                                                                    )
                                                                )
                    )


#####Uniendo los dataframe de las diferentes transacciones

In [0]:
trx_bronze_df = app_df_agg.unionByName(web_df_agg, allowMissingColumns=True) \
                          .unionByName(atm_df_agg, allowMissingColumns=True)

#####Agregando las columnas fecha_auditoria y timestamp_auditoria al dataframe generado

In [0]:
trx_bronze_df = trx_bronze_df.withColumn("fecha_auditoria", current_date()) \
                            .withColumn("timestamp_auditoria", current_timestamp())

#####Generando la tabla delta trx_bronze

In [0]:
trx_bronze_df.write.format("delta").mode("overwrite").saveAsTable("catalgbanco.capabronze.trx_bronze")

#####Optimizando la tabla delta

In [0]:
spark.sql("OPTIMIZE catalgbanco.capabronze.trx_bronze")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

####Moviendo todos los archivos .json a otra carpeta respaldo
#####De esta manera quedaran vacias las carpetas en espera de otra ingesta de lotes de archivos
#####y así solo se cargarán los nuevos archivos sin necesidad de cargar los anteriores

In [0]:
dbutils.fs.mv("dbfs:/temp/trx_app/*.json", "dbfs:/respaldo/trx_app")
dbutils.fs.mv("dbfs:/temp/trx_web/*.json", "dbfs:/respaldo/trx_web")
dbutils.fs.mv("dbfs:/temp/trx_atm/*.json", "dbfs:/respaldo/trx_atm")